<a href="https://colab.research.google.com/github/sanvirrafsaan/Data-Science-Portfolio/blob/main/Churn-prediction/notebooks/02_eda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd

from statsmodels.stats.outliers_influence import variance_inflation_factor

In [9]:
import os
from pathlib import Path

# --- Detect environment ---
IN_COLAB = False
try:
    from google.colab import drive
    IN_COLAB = True
except ImportError:
    pass

# --- Project root ---
if IN_COLAB:
    drive.mount('/content/drive')
    PROJECT_ROOT = Path("/content/drive/MyDrive/Data Science Portfolio/churn-prediction")
else:
    PROJECT_ROOT = Path("/Users/user/Desktop/data-science-projects/Data-Science-Portfolio/churn-prediction")

# --- Paths ---
RAW_DIR = PROJECT_ROOT / "data" / "raw"
INTERIM_DIR = PROJECT_ROOT / "data" / "interim"
PROCESSED_DIR = PROJECT_ROOT / "data" / "processed"
MODELS_DIR = PROJECT_ROOT / "models"
FIGURES_DIR = PROJECT_ROOT / "reports" / "figures"

# --- Ensure directories exist ---
for p in [RAW_DIR, INTERIM_DIR, PROCESSED_DIR, MODELS_DIR, FIGURES_DIR]:
    p.mkdir(parents=True, exist_ok=True)

# --- Debug ---
print("Running in Colab:", IN_COLAB)
print("Project root:", PROJECT_ROOT)
print("Interim contents:", [x.name for x in INTERIM_DIR.iterdir()])


Running in Colab: False
Project root: /Users/user/Desktop/data-science-projects/Data-Science-Portfolio/churn-prediction
Interim contents: []
